### Data 620 | Project 3

#### Team 3: Scott Karr, Omar Pineda Jr., Vikas Sinha, Aryeh Sturm, Antonio Bayquen

https://www.nltk.org/book/ch06.html

#### This work is a recitation of the gender classification exercise in Chapter 6 of Natural Language Processing with Python along with revisions to the gender features extractor which improve performance on the test set to 82.4%.


# Requirements:
Project 3 - This is a Team Project!
<center>------------------------------------------------------------------------------------------- o ----------------------------------------------------------------------------------------</center>
For this project, please work with the entire class as one collaborative group! Your project should be submitted (as a Jupyter Notebook via GitHub) by end of the due date. The group should present their code and findings in our meetup.

The ability to be an effective member of a virtual team is highly valued in the data science job market.
<center>------------------------------------------------------------------------------------------- o ----------------------------------------------------------------------------------------</center>


Using any of the three classifiers described in chapter 6 of Natural Language Processing with Python, and any features you can think of, build the best name gender classifier you can.

Begin by splitting the Names Corpus into three subsets: 

        (1) 500 words for the test set
        (2) 500 words for the dev- test set, and 
        (3) the remaining 6900 words for the training set 
        
Then, starting with the example name gender classifier, make incremental improvements. Use the dev-test set to check your progress. Once you are satisfied with your classifier, check its final performance on the test set.

How does the performance on the test set compare to the performance on the dev-test set? Is this what you'd expect?
Source: Natural Language Processing with Python, exercise 6.10.2.

# <center>6. Learning to Classify Text</center>

Detecting patterns is a central part of Natural Language Processing. Words ending in "-ed" tend to be past tense verbs (5.). Frequent use of "will" is indicative of news text (3). These observable patterns — word structure and word frequency — happen to correlate with particular aspects of meaning, such as tense and topic. But how did we know where to start looking, which aspects of form to associate with which aspects of meaning?

The goal of this chapter is to answer the following questions:

    (1) How can we identify particular features of language data that are salient for classifying it?
    (2) How can we construct models of language that can perform language processing tasks automatically?
    (3) What can we learn about language from these models?

Along the way we will study some important machine learning techniques, including decision trees, naive Bayes' classifiers, and maximum entropy classifiers. We will gloss over the mathematical and statistical underpinnings of these techniques, focusing instead on how and when to use them (see the Further Readings section for more technical background). Before looking at these methods, we first need to appreciate the broad scope of this topic.

# 1   Supervised Classification
Classification is the task of choosing the correct class label for a given input. In basic classification tasks, each input is considered in isolation from all other inputs, and the set of labels is defined in advance. Some examples of classification tasks are:

Deciding whether an email is spam or not.
Deciding what the topic of a news article is, from a fixed list of topic areas such as "sports," "technology," and "politics."
Deciding whether a given occurrence of the word bank is used to refer to a river bank, a financial institution, the act of tilting to the side, or the act of depositing something in a financial institution.
The basic classification task has a number of interesting variants. For example, in multi-class classification, each instance may be assigned multiple labels; in open-class classification, the set of labels is not defined in advance; and in sequence classification, a list of inputs are jointly classified.

A classifier is called supervised if it is built based on training corpora containing the correct label for each input. The framework used by supervised classification is shown in 1.1.

![supervised-classification.png](https://raw.githubusercontent.com/vsinha-cuny/data620Proj3/master/supervised-classification.png)

Figure 1.1: Supervised Classification. 

    (a) During training, a feature extractor is used to convert each input value to a feature set. These 
        feature sets, which capture the basic information about each input that should be used to classify it,  
        are discussed in the next section. Pairs of feature sets and labels are fed into the machine learning        
        algorithm to generate a model. 

    (b) During prediction, the same feature extractor is used to convert unseen inputs to feature sets. These 
        feature sets are then fed into the model, which generates predicted labels.

In the rest of this section, we will look at how classifiers can be employed to solve a wide variety of tasks. Our discussion is not intended to be comprehensive, but to give a representative sample of tasks that can be performed with the help of text classifiers.

# 1.1   Gender Identification

Male and female names have some distinctive characteristics. Names ending in a, e and i are likely to be female, while names ending in k, o, r, s and t are likely to be male. Let's build a classifier to model these differences more precisely.

The first step in creating a classifier is deciding what features of the input are relevant, and how to encode those features. For this example, we'll start by just looking at the final letter of a given name. The following feature extractor function builds a dictionary containing relevant information about a given name:

In [70]:
import nltk
import random
import numpy as np

#nltk.download('names')
from nltk.corpus import names

In [71]:
def gender_features(name):
    name=name.upper()
    return {
        'last_letter': name[-1]
    }

In [72]:
gender_features('Shrek')

{'last_letter': 'K'}

Now that we've defined a feature extractor, we need to prepare a list of examples and corresponding class labels.

In [73]:
# get list of ordered names from nltk.corpus by importing names
labeled_names = (
                    [(name, 'male') for name in names.words('male.txt')] +
                    [(name, 'female') for name in names.words('female.txt')]
)

np.random.seed(222)
# randomized the name orders
random.shuffle(labeled_names)
print(len(labeled_names))

7944


Next, we use the feature extractor to process the names data, and divide the resulting list of feature sets into a training set and a test set. The training set is used to train a new "naive Bayes" classifier.

In [75]:
np.random.seed(222)

# featuresets, train_set & test_set are dictionaries containing 2 elements per record, 
#  - the gender function return value &
#  - the actual gender from the labeled_names set that was imported
featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]

#  apply Naive Bayes guess to train set and check classifier's predictions
classifier = nltk.NaiveBayesClassifier.train(train_set)

We will learn more about the naive Bayes classifier later in the chapter. For now, let's just test it out on some names that did not appear in its training data:

In [76]:
# use trained classifier to predict 2 names gender based upon last character
print('Neo is: ' + classifier.classify(gender_features('Neo')))
print('Trinity is: ' + classifier.classify(gender_features('Trinity')))

Neo is: male
Trinity is: female


Observe that these character names from The Matrix are correctly classified. Although this science fiction movie is set in 2199, it still conforms with our expectations about names and genders. We can systematically evaluate the classifier on a much larger quantity of unseen data:



In [77]:
print(nltk.classify.accuracy(classifier, test_set))

0.78


Finally, we can examine the classifier to determine which features it found most effective for distinguishing the names' genders:

In [78]:
classifier.show_most_informative_features(5)

Most Informative Features
             last_letter = 'A'            female : male   =     37.2 : 1.0
             last_letter = 'K'              male : female =     30.5 : 1.0
             last_letter = 'P'              male : female =     18.5 : 1.0
             last_letter = 'F'              male : female =     14.4 : 1.0
             last_letter = 'V'              male : female =     11.1 : 1.0


This listing shows that the names in the training set that end in "a" are female 33 times more often than they are male, but names that end in "k" are male 32 times more often than they are female. These ratios are known as likelihood ratios, and can be useful for comparing different feature-outcome relationships.

When working with large corpora, constructing a single list that contains the features of every instance can use up a large amount of memory. In these cases, use the function nltk.classify.apply_features, which returns an object that acts like a list but does not store all the feature sets in memory:

In [79]:
from nltk.classify import apply_features
train_set = apply_features(gender_features, labeled_names[500:])
test_set = apply_features(gender_features, labeled_names[:500])

Typically, feature extractors are built through a process of trial-and-error, guided by intuitions about what information is relevant to the problem. It's common to start with a "kitchen sink" approach, including all the features that you can think of, and then checking to see which features actually are helpful. 

In [80]:
def gender_features2(name):
    features = {}
    features["first_letter"] = name[:1].lower()
    features["first_two"] = name[:2].lower()   
    features["first_three"] = name[:3].lower()      
    features["last_letter"] = name[-1].lower()
    features["last_two"] = name[-2:].lower()     
    features["suffix3"] = name[-3:].lower()
    features["suffix4"] = name[-4:].lower()    
    features["last_is_vowel"] = (name[-1].lower() in 'aeiouy')
    features["odd"] = (name.find("-") > 0),     
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

In [81]:
# featuresets, train_set & test_set are dictionaries containing 2 elements per record, 
#  - the gender function return value &
#  - the actual gender from the labeled_names set that was imported
featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]

#  apply Naive Bayes guess to train set and check classifier's predictions
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [82]:
classifier.show_most_informative_features(50)

Most Informative Features
                last_two = 'na'           female : male   =     99.7 : 1.0
                last_two = 'la'           female : male   =     78.1 : 1.0
                last_two = 'us'             male : female =     40.6 : 1.0
                last_two = 'ia'           female : male   =     40.6 : 1.0
             last_letter = 'a'            female : male   =     37.2 : 1.0
             last_letter = 'k'              male : female =     30.5 : 1.0
                last_two = 'do'             male : female =     27.0 : 1.0
                last_two = 'ra'           female : male   =     26.7 : 1.0
                 suffix3 = 'ana'          female : male   =     26.5 : 1.0
                 suffix4 = 'bert'           male : female =     26.1 : 1.0
                last_two = 'rd'             male : female =     25.5 : 1.0
                last_two = 'ta'           female : male   =     25.2 : 1.0
                last_two = 'io'             male : female =     24.8 : 1.0

In [83]:
# Extract features for the Isabelle as an example
# gender_features2('Isabelle') 

However, there are usually limits to the number of features that you should use with a given learning algorithm — if you provide too many features, then the algorithm will have a higher chance of relying on idiosyncrasies of your training data that don't generalize well to new examples. This problem is known as overfitting, and can be especially problematic when working with small training sets. For example, if we train a naive Bayes classifier using the feature extractor shown in 1.2, it will overfit the relatively small training set, resulting in a system whose accuracy is about 1% lower than the accuracy of a classifier that only pays attention to the final letter of each name:

In [84]:
np.random.seed(224)

featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.83


Once an initial set of features has been chosen, a very productive method for refining the feature set is error analysis. First, we select a development set, containing the corpus data for creating the model. This development set is then subdivided into the training set and the dev-test set.

In [85]:
train_names = labeled_names[1500:]
devtest_names = labeled_names[500:1500]
test_names = labeled_names[:500]

The training set is used to train the model, and the dev-test set is used to perform error analysis. The test set serves in our final evaluation of the system. For reasons discussed below, it is important that we employ a separate dev-test set for error analysis, rather than just using the test set. The division of the corpus data into different subsets is shown in 1.3.



![corpus-org.png](https://raw.githubusercontent.com/vsinha-cuny/data620Proj3/master/corpus-org.png)

Figure 1.3: Organization of corpus data for training supervised classifiers. The corpus data is divided into two sets: the development set, and the test set. The development set is often further subdivided into a training set and a dev-test set.

Having divided the corpus into appropriate datasets, we train a model using the training set [1], and then run it on the dev-test set [2].

In [88]:
np.random.seed(225)
train_set = [(gender_features2(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features2(n), gender) for (n, gender) in devtest_names]
test_set = [(gender_features2(n), gender) for (n, gender) in test_names]
classifier = nltk.NaiveBayesClassifier.train(train_set) # [1]
print(nltk.classify.accuracy(classifier, devtest_set)) # [2]

0.831


We also considered using a Decision Tree classifier, but proceeded with the Naive Bayes classifier instead due to its greater accuracy.

![decision-tree.png](https://raw.githubusercontent.com/vsinha-cuny/data620Proj3/master/decision-tree.png)

In [89]:
classifier2 = nltk.DecisionTreeClassifier.train(train_set)
print(f'Decision Tree Accuracy: {nltk.classify.accuracy(classifier2, devtest_set)}')

Decision Tree Accuracy: 0.689


In [91]:
classifier3 = nltk.MaxentClassifier.train(train_set)
print(f'Maximum entropy Accuracy: {nltk.classify.accuracy(classifier3, devtest_set)}')

  ==> Training (100 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -0.69315        0.373
             2          -0.58801        0.627
             3          -0.55283        0.634
             4          -0.52182        0.697
             5          -0.49454        0.763
             6          -0.47054        0.801
             7          -0.44937        0.826
             8          -0.43063        0.843
             9          -0.41396        0.854
            10          -0.39907        0.861
            11          -0.38570        0.867
            12          -0.37364        0.873
            13          -0.36271        0.877
            14          -0.35275        0.881
            15          -0.34365        0.883
            16          -0.33530        0.886
            17          -0.32760        0.889
            18          -0.32048        0.891
            19          -0.31388        0.893
 

Using the dev-test set, we can generate a list of the errors that the classifier makes when predicting name genders:

In [27]:
errors = []
for (name, tag) in devtest_names:
    guess = classifier.classify(gender_features2(name))
    if guess != tag:
        errors.append( (tag, guess, name)
) 

We can then examine individual error cases where the model predicted the wrong label, and try to determine what additional pieces of information would allow it to make the right decision (or which existing pieces of information are tricking it into making the wrong decision). The feature set can then be adjusted accordingly. The names classifier that we have built generates about 100 errors on the dev-test corpus:



In [32]:
for (tag, guess, name) in sorted(errors):
    print('correct={:<8} guess={:<8s} name={:<30}'.format(tag, guess, name))

correct=female   guess=male     name=Angil                         
correct=female   guess=male     name=Ariel                         
correct=female   guess=male     name=Auguste                       
correct=female   guess=male     name=Barry                         
correct=female   guess=male     name=Beitris                       
correct=female   guess=male     name=Bird                          
correct=female   guess=male     name=Bren                          
correct=female   guess=male     name=Bridgett                      
correct=female   guess=male     name=Brook                         
correct=female   guess=male     name=Brynn                         
correct=female   guess=male     name=Buffy                         
correct=female   guess=male     name=Calypso                       
correct=female   guess=male     name=Camel                         
correct=female   guess=male     name=Chad                          
correct=female   guess=male     name=Charlott   

Looking through this list of errors makes it clear that some suffixes that are more than one letter can be indicative of name genders. For example, names ending in yn appear to be predominantly female, despite the fact that names ending in n tend to be male; and names ending in ch are usually male, even though names that end in h tend to be female. We therefore adjust our feature extractor to include features for two-letter suffixes:



In [33]:
def gender_features(word):
    return {'suffix1': word[-1:],'suffix2': word[-2:]}

Rebuilding the classifier with the new feature extractor, we see that the performance on the dev-test dataset improves by almost 2 percentage points (from 76.5% to 78.2%):

In [34]:
train_set = [(gender_features(n), gender) for (n, gender) in train_names]
devtest_set = [(gender_features(n), gender) for (n, gender) in devtest_names]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, devtest_set))

0.781


This error analysis procedure can then be repeated, checking for patterns in the errors that are made by the newly improved classifier. Each time the error analysis procedure is repeated, we should select a different dev-test/training split, to ensure that the classifier does not start to reflect idiosyncrasies in the dev-test set.

But once we've used the dev-test set to help us develop the model, we can no longer trust that it will give us an accurate idea of how well the model would perform on new data. It is therefore important to keep the test set separate, and unused, until our model development is complete. At that point, we can use the test set to evaluate how well our model will perform on new input values.